In [182]:
import os
os.chdir('../')

from extract import open_server_users
from extract import open_server_engage
from extract import open_server_payment
from extract import open_server_battle
from extract import open_server_resource
from extract import open_server_troop
from extract import server_open_nd
from extract import open_server_goldfarmer

from transform import gold_farmer_aggr

import numpy as np
from config import ETLConfig
from functools import reduce

In [205]:
task = dict(game = 'aoz',server_ids = [234,235,236],
            offset = ETLConfig.OFFSET,days = ETLConfig.OPEN_SERVER_DAYS)

task['server_open'] = server_open_nd(**task)

user = open_server_users(**task, fields = ETLConfig.USER_INFO_FIELDS)

engage = open_server_engage(**task, fields = ETLConfig.ENGAGE_FIELDS)

payment = open_server_payment(**task, fields = ETLConfig.PAYMENT_FIELDS)

battle = open_server_battle(**task, fields = ETLConfig.BATTLE_FIELDS)

resource = open_server_resource(**task, fields = ETLConfig.RESOURCE_FIELDS)

troop = open_server_troop(**task,fields = ETLConfig.TROOP_FIELDS)

goldfarmer = open_server_goldfarmer(**task, fields = ETLConfig.GOLD_FARMER_FIELDS)

gold_farmer = gold_farmer_aggr(goldfarmer, ETLConfig)  

GoldFarmerRob 236: 100%|████████████████████████████████████| 3/3 [01:06<00:00, 22.27s/it]


In [214]:
payment.groupby()

,user_id,create_time,amount,type,from_server,date,day_id
0,19692246,2020-05-03 04:59:31.664,1.99,19,234,2020-05-03,1
1,19736925,2020-05-03 18:04:26.687,9.99,32,234,2020-05-03,1
2,19736925,2020-05-03 18:05:02.719,1.99,19,234,2020-05-03,1
3,19736925,2020-05-03 19:53:50.058,3.99,24,234,2020-05-03,1
4,19736925,2020-05-03 19:54:46.201,1.99,33,234,2020-05-03,1
...,...,...,...,...,...,...,...
8012,19933822,2020-05-11 23:42:52.480,1.99,34,236,2020-05-11,6
8013,19933822,2020-05-11 23:43:42.702,1.99,33,236,2020-05-11,6
8014,19923302,2020-05-11 23:45:05.099,1.99,19,236,2020-05-11,6
8015,19942137,2020-05-11 23:45:21.859,1.99,13,236,2020-05-11,6


In [224]:
PAYMENT_TO_DAILY = (
    #  group by index          # columns         # calc val   # method  # missing fill    
    (['from_server','day_id','user_id'],[],                      ['amount'], [np.sum],0), 
    (['from_server','day_id','user_id'],['type'],                ['amount'], [np.sum],0),
    (['from_server','day_id','user_id'],[],                      ['amount'], ['count'],0),
    (['from_server','day_id','user_id'],['type'],                ['amount'], ['count'],0),
)

In [226]:
payment = run_groupby_tasks(payment, PAYMENT_TO_DAILY,merge= True).reset_index()

,from_server,day_id,user_id,sum_amount,type_1_sum_amount,type_2_sum_amount,type_3_sum_amount,type_10_sum_amount,type_11_sum_amount,type_12_sum_amount,...,type_24_count_amount,type_25_count_amount,type_26_count_amount,type_27_count_amount,type_28_count_amount,type_29_count_amount,type_31_count_amount,type_32_count_amount,type_33_count_amount,type_34_count_amount
0,234,1,19620894,30.94,0.0,0.00,0.00,11.98,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,234,1,19668094,98.84,0.0,19.99,9.99,8.99,0.0,14.98,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
2,234,1,19668146,21.98,0.0,19.99,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,234,1,19669223,42.94,0.0,0.00,9.99,8.99,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,234,1,19671161,24.98,0.0,19.99,0.00,4.99,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,236,6,20060276,16.97,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3092,236,6,20101233,8.97,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3093,236,6,20101595,19.99,0.0,19.99,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3094,236,6,20114852,9.99,0.0,0.00,9.99,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
def run_groupby_tasks(df, tasks, merge = False):
    
    dfs = []
    for ind, cols, vals, methods, na in tasks:

        piv = df.pivot_table(index = ind, 
                     columns = cols, 
                     values = vals, 
                     aggfunc=methods).fillna(na)

        if piv.columns.names[-1]:
            names = []
            for op_type, agg_field, by in piv.columns:
                name = f'{piv.columns.names[-1]}_{by}_{op_type}_{agg_field}'
                names.append(name)
            piv.columns = names

        else:
            piv.columns = ['_'.join(piv.columns[0])]

        dfs.append(piv)
        
    if merge:
        return reduce(lambda x,y:x.merge(y, left_index = True, right_index = True, how = 'left'), dfs)
    return dfs

In [30]:
user.columns

Index(['user_id', 'create_time', 'language', 'register_channel',
       'login_channel', 'country_code', 'register_ip_country', 'bind_channel',
       'device_id', 'device_model', 'source_channel', 'from_server', 'date',
       'user_type', 'day_id'],
      dtype='object')

In [208]:
USER_TASKS = (
    #  group by index          # columns         # calc val   # method  # missing fill    
    (['from_server','day_id'],[],                ['user_id'], ['count'],0),
    (['from_server','day_id'],['language'],      ['user_id'], ['count'],0),
    (['from_server','day_id'],['source_channel'],['user_id'], ['count'],0),
    (['from_server','day_id'],['user_type'],     ['user_id'], ['count'],0),
    (['from_server','day_id'],['device_model'],  ['user_id'], ['count'],0),
    (['from_server','day_id'],['bind_channel'],  ['user_id'], ['count'],0),   
)

In [210]:
USER_TASKS

((['from_server', 'day_id'], [], ['user_id'], ['count'], 0),
 (['from_server', 'day_id'], ['language'], ['user_id'], ['count'], 0),
 (['from_server', 'day_id'], ['source_channel'], ['user_id'], ['count'], 0),
 (['from_server', 'day_id'], ['user_type'], ['user_id'], ['count'], 0),
 (['from_server', 'day_id'], ['device_model'], ['user_id'], ['count'], 0),
 (['from_server', 'day_id'], ['bind_channel'], ['user_id'], ['count'], 0))

In [209]:
user_features = run_groupby_tasks(user, USER_TASKS, merge = True)
user_features

count_user_id  language_1_count_user_id  \
from_server day_id                                            
234         1                6730                     120.0   
            2               48518                    1861.0   
            3               14478                     942.0   
            4                 389                     214.0   
            5                 135                      13.0   
            6                  91                      12.0   
235         1               34540                    1123.0   
            2               36972                    1978.0   
            3                 289                      17.0   
            4                 168                      11.0   
            5                 133                       7.0   
            6                  90                       3.0   
236         1               13670                     243.0   
            2               49517                    1906.0   
            3               11893                     602.0   
            4                 173                       8.0   
            5                 110                       6.0   
            6                  81                       6.0   

                    language_2_count_user_id  language_3_count_user_id  \
from_server day_id                                                       
234         1                           25.0                    2705.0   
            2                          426.0                   15734.0   
            3                          140.0                    4755.0   
            4                            1.0                      77.0   
            5                            0.0                      36.0   
            6                            2.0                      30.0   
235         1                          287.0                   11326.0   
            2                          365.0                   11376.0   
            3                            4.0                      82.0   
            4                            9.0                      35.0   
            5                            3.0                      27.0   
            6                            0.0                      22.0   
236         1                           72.0                    5278.0   
            2                          407.0                   16149.0   
            3                          113.0                    4150.0   
            4                            0.0                      61.0   
            5                            0.0                      42.0   
            6                            0.0                      25.0   

                    language_5_count_user_id  language_6_count_user_id  \
from_server day_id                                                       
234         1                          198.0                     134.0   
            2                         2814.0                    2003.0   
            3                         1228.0                     907.0   
            4                           12.0                       4.0   
            5                           14.0                       3.0   
            6                            5.0                       1.0   
235         1                         1715.0                    1465.0   
            2                         2848.0                    1359.0   
            3                            5.0                      15.0   
            4                           17.0                       6.0   
            5                           10.0                      12.0   
            6                            7.0                       3.0   
236         1                          231.0                     154.0   
            2                         1935.0                    1395.0   
            3                          594.0                     370.0   
            4       

In [194]:
engage_tasks = (
    #  group by index          # columns         # calc val   # method  # missing fill    
    (['from_server','day_id'],  [],              ['active_time'], [np.max],0),   
    (['from_server','day_id'],  [],              ['active_time'], [np.mean],0),
    (['from_server','day_id'],  [],              ['active_time'], [np.std],0),
    (['from_server','day_id'],  [],              ['active_time'], [np.median],0),
)

In [196]:
engage_features = run_groupby_tasks(engage, engage_tasks, merge = True)
engage_features

amax_active_time  mean_active_time  std_active_time  \
from_server day_id                                                        
234         1                  10582       1009.347025      1455.588251   
            2                  81925       1941.037746      3883.854457   
            3                  76491       3161.116944      5338.157017   
            4                  81602       4580.092325      6347.743021   
            5                  84904       4546.622220      6272.693177   
            6                  78456       4689.918668      6293.958004   
235         1                 116789       1644.325494      3321.159885   
            2                  72351       2334.240039      4446.001852   
            3                  80931       4159.514400      6182.986136   
            4                  78824       4349.455663      6225.416107   
            5                  79115       4605.895933      6652.730051   
            6                  73875       4814.317105      6701.866088   
236         1                  20173       1187.787391      1982.209890   
            2                  58133       1838.466970      3554.709832   
            3                  79514       2958.597286      4926.595876   
            4                  78961       4053.601242      5972.866959   
            5                  68409       4290.238657      6281.866815   
            6                  79117       4445.879072      6322.196415   

                    median_active_time  
from_server day_id                      
234         1                    415.0  
            2                    542.0  
            3                   1010.5  
            4                   2144.0  
            5                   2113.0  
            6                   2329.0  
235         1                    493.0  
            2                    661.0  
            3                   1964.0  
            4                   2193.0  
            5                   2112.0  
            6                   2332.0  
236         1                    422.5  
            2                    523.0  
            3                    992.0  
            4                   1814.0  
            5                   1991.0  
            6                   2098.0

In [197]:
payment.columns

Index(['user_id', 'create_time', 'amount', 'type', 'from_server', 'date',
       'day_id'],
      dtype='object')

### 错了，忘了聚合了。。。宛如智障

In [198]:
payment_tasks = (
    #  group by index          # columns         # calc val   # method  # missing fill    
    (['from_server','day_id'],  [],              ['amount'], [np.max],0),   
    (['from_server','day_id'],  [],              ['amount'], [np.mean],0),
    (['from_server','day_id'],  [],              ['amount'], [np.std],0),
    (['from_server','day_id'],  [],              ['amount'], [np.median],0),
)

In [199]:
payment_features = run_groupby_tasks(payment, payment_tasks, merge = True)
payment_features

amax_amount  mean_amount  std_amount  median_amount
from_server day_id                                                     
234         1             19.99     6.429655    5.592369           4.99
            2             49.99     5.612673    5.332481           4.99
            3             99.99     5.312222    5.732454           4.99
            4             99.99     8.630777   11.169767           4.99
            5             99.99     6.679320    8.048285           4.99
            6             99.99     7.129875    7.540356           4.99
235         1             19.99     4.782041    4.626092           2.99
            2             99.99     7.058836   11.114941           4.99
            3             99.99     7.577751   11.467783           4.99
            4             99.99     8.466440   14.340560           4.99
            5             99.99     6.648974    7.102924           4.99
            6             99.99     7.778177   10.430698           4.99
236         1             49.99     7.672927    6.575429           4.99
            2             99.99     7.203198    9.700178           4.99
            3             99.99     7.023384    9.687461           4.99
            4             99.99     8.781075   13.840370           4.99
            5             99.99     9.194348   13.121254           4.99
            6             99.99     8.764262   14.935297           4.99

In [200]:
payment

,user_id,create_time,amount,type,from_server,date,day_id
0,19692246,2020-05-03 04:59:31.664,1.99,19,234,2020-05-03,1
1,19736925,2020-05-03 18:04:26.687,9.99,32,234,2020-05-03,1
2,19736925,2020-05-03 18:05:02.719,1.99,19,234,2020-05-03,1
3,19736925,2020-05-03 19:53:50.058,3.99,24,234,2020-05-03,1
4,19736925,2020-05-03 19:54:46.201,1.99,33,234,2020-05-03,1
...,...,...,...,...,...,...,...
8012,19933822,2020-05-11 23:42:52.480,1.99,34,236,2020-05-11,6
8013,19933822,2020-05-11 23:43:42.702,1.99,33,236,2020-05-11,6
8014,19923302,2020-05-11 23:45:05.099,1.99,19,236,2020-05-11,6
8015,19942137,2020-05-11 23:45:21.859,1.99,13,236,2020-05-11,6
